In [93]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import pickle
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from sklearn.metrics import roc_curve, auc
from scipy import interp

rcParams['figure.figsize'] = 12, 4

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
data= pickle.load(open('/Users/chaitanya/Documents/python/keystrokes/data_augmentation/greyc1/PCA_GREYC_2X.pickle','rb'))


In [97]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t

In [151]:
def GetEER_(y_score, y_test):
	n_classes = y_score.shape[1]
	fpr = dict()
	tpr = dict()
	roc_auc = dict()
	missRate = dict()
	for i in range(n_classes):
		fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
		roc_auc[i] = auc(fpr[i], tpr[i])
	fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
	missRate["micro"] = 1 - tpr["micro"]
	all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
	# Then interpolate all ROC curves at this points
	mean_tpr = np.zeros_like(all_fpr)
	for i in range(n_classes):
		mean_tpr += interp(all_fpr, fpr[i], tpr[i])
	mean_tpr /= n_classes
	fpr["macro"] = all_fpr
	tpr["macro"] = mean_tpr
	missRate["macro"] = 1 - tpr["macro"]
	return  min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])


In [161]:
def GetEER(y_score, y_test):
    #y_score = y_score.get_label()
    y_test = np.array(y_test.get_label()).reshape(len(y_score), 1)
    n_classes = y_score.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    missRate = dict()
    y_test_ = [onehot(int(x+1), 100) for x in y_test]
    y_test = np.reshape(y_test_, (len(y_test_),100))
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    missRate["micro"] = 1 - tpr["micro"]
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    missRate["macro"] = 1 - tpr["macro"]
    #print(np.dtype(min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])))
    return 'EER', min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])


In [128]:
X = data['data']
X = np.array(X).reshape(len(X), 28)
y = np.array(data['labels']).reshape(len(X), 100)
#X = X[:100,:]
#y = y[:100,:]
#y_train = [onehot(x,100) for x in train_data['labels']]
#X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
#y_test = [onehot(x,100) for x in test_data['labels']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=12)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=122)
print([len(x) for x in [X_test, y_test, X_train, y_train]])

#y_train=np.reshape(y_train,(len(y_train)*100))
#y_test = np.reshape(y_test, (len(y_test)*100))

[2223, 2223, 12593, 12593]


In [162]:
y_train_ = [np.argmax(x) for x in y_train]
y_test_ = [np.argmax(x) for x in y_test]
xg_train = xgb.DMatrix(X_train, label = y_train_)
xg_test = xgb.DMatrix(X_test, label = y_test_)
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'eval_metric': 'merror',
    'min_child_weight': 1,
    'eta': 0.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'multi:softprob',
    'num_class' : 100,
}
num_boost_round = 999
params['tree_method']= 'hist'

In [163]:
model = xgb.train(
    params,
    xg_train,
    num_boost_round=num_boost_round,
    evals=[(xg_test, "Test")],
    early_stopping_rounds=10,
    feval = GetEER,
    
)

[0]	Test-merror:0.299595	Test-EER:0.133185
Multiple eval metrics have been passed: 'Test-EER' will be used for early stopping.

Will train until Test-EER hasn't improved in 10 rounds.
[1]	Test-merror:0.277103	Test-EER:0.109262
[2]	Test-merror:0.255511	Test-EER:0.099347
[3]	Test-merror:0.238866	Test-EER:0.087369
[4]	Test-merror:0.22897	Test-EER:0.080917
[5]	Test-merror:0.221772	Test-EER:0.074265
[6]	Test-merror:0.213675	Test-EER:0.072225
[7]	Test-merror:0.210526	Test-EER:0.069262
[8]	Test-merror:0.205128	Test-EER:0.066345
[9]	Test-merror:0.20108	Test-EER:0.062655
[10]	Test-merror:0.197931	Test-EER:0.060879
[11]	Test-merror:0.191633	Test-EER:0.060345
[12]	Test-merror:0.190283	Test-EER:0.058402
[13]	Test-merror:0.185335	Test-EER:0.057107
[14]	Test-merror:0.184435	Test-EER:0.055153
[15]	Test-merror:0.180387	Test-EER:0.05477
[16]	Test-merror:0.177688	Test-EER:0.052872
[17]	Test-merror:0.174989	Test-EER:0.052441
[18]	Test-merror:0.17139	Test-EER:0.050923
[19]	Test-merror:0.17049	Test-EER:0.0

In [164]:
pred = model.predict(xg_test)
GetEER_(pred,y_test)

0.04217687074829932

In [166]:
cv_results = xgb.cv(
    params,
    xg_train,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'merror'},
    early_stopping_rounds=10,
    feval = GetEER,
)


KeyboardInterrupt: 

In [46]:
cv_results

,train-merror-mean,train-merror-std,test-merror-mean,test-merror-std
0,0.219706,0.001887,0.302707,0.011303
1,0.166938,0.002620,0.285238,0.009576
2,0.126916,0.001585,0.264591,0.008174
3,0.104324,0.002049,0.246089,0.006187
4,0.089216,0.002075,0.234574,0.006100
5,0.073910,0.001888,0.226554,0.008428
6,0.060073,0.001529,0.217978,0.008516
7,0.046454,0.001211,0.213769,0.007248
8,0.034960,0.000816,0.209640,0.006373
9,0.026086,0.001223,0.204717,0.006018


In [168]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

In [169]:
min_merror = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))

    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight

    # Run CV
    cv_results = xgb.cv(
        params,
        xg_train,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'merror'},
        early_stopping_rounds=10,
        feval = GetEER,
        
    )

    # Update best MAE
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMerror {} for {} rounds".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_merror
        best_params = (max_depth,min_child_weight)

print("Best params: {}, {}, Merror: {}".format(best_params[0], best_params[1], min_merror))

CV with max_depth=9, min_child_weight=5


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.122211 for 119 rounds
CV with max_depth=9, min_child_weight=6


KeyboardInterrupt: 

In [71]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(10,13)
    for min_child_weight in range(7,11)
]
min_merror = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))

    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight

    # Run CV
    cv_results = xgb.cv(
        params,
        xg_train,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'merror'},
        early_stopping_rounds=10,
        
    )

    # Update best MAE
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMerror {} for {} rounds".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_merror
        best_params = (max_depth,min_child_weight)

print("Best params: {}, {}, Merror: {}".format(best_params[0], best_params[1], min_merror))

CV with max_depth=10, min_child_weight=7


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.1170488 for 71 rounds
CV with max_depth=10, min_child_weight=8
	Merror 0.1141906 for 93 rounds
CV with max_depth=10, min_child_weight=9
	Merror 0.1146672 for 76 rounds
CV with max_depth=10, min_child_weight=10
	Merror 0.1146674 for 61 rounds
CV with max_depth=11, min_child_weight=7
	Merror 0.1170488 for 71 rounds
CV with max_depth=11, min_child_weight=8
	Merror 0.1141906 for 93 rounds
CV with max_depth=11, min_child_weight=9
	Merror 0.1146672 for 76 rounds
CV with max_depth=11, min_child_weight=10
	Merror 0.1146674 for 61 rounds
CV with max_depth=12, min_child_weight=7
	Merror 0.1170488 for 71 rounds
CV with max_depth=12, min_child_weight=8
	Merror 0.1141906 for 93 rounds
CV with max_depth=12, min_child_weight=9
	Merror 0.1146672 for 76 rounds
CV with max_depth=12, min_child_weight=10
	Merror 0.1146674 for 61 rounds
Best params: 10, 8, Merror: 0.1141906


In [73]:
params['max_depth'] = 10
params['min_child_weight'] = 8

In [74]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [75]:
min_merror = float("Inf")
best_params = None

# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))

    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample

    # Run CV
    cv_results = xgb.cv(
        params,
        xg_train,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'merror'},
        early_stopping_rounds=10
    )

    # Update best score
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMerror {} for {} rounds".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_merror
        best_params = (subsample,colsample)

print("Best params: {}, {}, Merror: {}".format(best_params[0], best_params[1], min_merror))

CV with subsample=1.0, colsample=1.0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.1141906 for 93 rounds
CV with subsample=1.0, colsample=0.9
	Merror 0.1158584 for 61 rounds
CV with subsample=1.0, colsample=0.8
	Merror 0.1166528 for 66 rounds
CV with subsample=1.0, colsample=0.7
	Merror 0.1137942 for 75 rounds
CV with subsample=0.9, colsample=1.0
	Merror 0.11403160000000001 for 48 rounds
CV with subsample=0.9, colsample=0.9
	Merror 0.11506419999999999 for 46 rounds
CV with subsample=0.9, colsample=0.8
	Merror 0.11387320000000001 for 72 rounds
CV with subsample=0.9, colsample=0.7
	Merror 0.1138738 for 89 rounds
CV with subsample=0.8, colsample=1.0
	Merror 0.11220539999999998 for 102 rounds
CV with subsample=0.8, colsample=0.9
	Merror 0.1123644 for 77 rounds
CV with subsample=0.8, colsample=0.8
	Merror 0.11236439999999999 for 77 rounds
CV with subsample=0.8, colsample=0.7
	Merror 0.1150646 for 64 rounds
CV with subsample=0.7, colsample=1.0
	Merror 0.114429 for 70 rounds
CV with subsample=0.7, colsample=0.9
	Merror 0.11514359999999998 for 47 rounds
CV with sub

In [77]:
params['subsample'] = 0.8
params['colsample_bytree'] = 1.0

In [80]:
min_merror = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01]:
    print("CV with eta={}".format(eta))

    # We update our parameters
    params['eta'] = eta
    %time
    # Run and time CV
    cv_results = xgb.cv(
            params,
            xg_train,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['merror'],
            early_stopping_rounds=10
          )

    # Update best score
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMerror {} for {} rounds\n".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_mae
        best_params = eta

print("Best params: {}, Merror: {}".format(best_params, min_merror))

CV with eta=0.3
CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 14.1 µs


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.11220539999999998 for 102 rounds

CV with eta=0.2
CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 12.2 µs


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.11037919999999998 for 79 rounds

CV with eta=0.1
CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10 µs


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.11093519999999998 for 140 rounds

CV with eta=0.05
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 11.9 µs


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	Merror 0.10990300000000001 for 245 rounds

CV with eta=0.01
CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 12.2 µs
	Merror 0.13563119999999998 for 403 rounds

Best params: 0.05, Merror: 0.12506940000000002


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


In [177]:
params['eta'] = 0.05
model = xgb.train(
    params,
    xg_train,
    num_boost_round = num_boost_round,
    evals = [(xg_test,"Test")],
    early_stopping_rounds =20
)

[0]	Test-merror:0.540261
Will train until Test-merror hasn't improved in 20 rounds.
[1]	Test-merror:0.383266
[2]	Test-merror:0.316239
[3]	Test-merror:0.272605
[4]	Test-merror:0.254611
[5]	Test-merror:0.236617
[6]	Test-merror:0.226271
[7]	Test-merror:0.220873
[8]	Test-merror:0.214575
[9]	Test-merror:0.212326
[10]	Test-merror:0.206478
[11]	Test-merror:0.202879
[12]	Test-merror:0.20018
[13]	Test-merror:0.197931
[14]	Test-merror:0.194782
[15]	Test-merror:0.194332
[16]	Test-merror:0.192533
[17]	Test-merror:0.189384
[18]	Test-merror:0.188484
[19]	Test-merror:0.185335
[20]	Test-merror:0.183986
[21]	Test-merror:0.181287
[22]	Test-merror:0.179487
[23]	Test-merror:0.176788
[24]	Test-merror:0.176788
[25]	Test-merror:0.174989
[26]	Test-merror:0.174089
[27]	Test-merror:0.173189
[28]	Test-merror:0.17004
[29]	Test-merror:0.17004
[30]	Test-merror:0.169591
[31]	Test-merror:0.168691
[32]	Test-merror:0.167791
[33]	Test-merror:0.166442
[34]	Test-merror:0.165542
[35]	Test-merror:0.165092
[36]	Test-merror:0

In [182]:
#ytest = np.reshape(y_test,(len(X_test), 100))
pred = model.predict(xg_test)
error_rate = np.sum(np.argmax(pred) != np.argmax(y_test))/np.shape(y_test)[0]

In [180]:
1-error_rate

0.9995501574448943

In [111]:
xg_test.get_label()

array([22., 62., 79., ...,  5., 48., 58.], dtype=float32)

In [183]:
GetEER_(pred,y_test)


0.020620055707775007

In [185]:
model.save_model('GREYC1.model')
